# Customer Lifetime and Customer Lifetime Value (CLTV)

This two concepts are related to each other but there are some differences that make them distinct.

### Customer Lifetime (or Customer Lifetime Span)
- refers to the duration time the customer continues to engage or transact with business.
- it represents the length of time from the first customer's interraction with the business to the last one.
- formula: Customer_Lifetime = (last_purchase_date - first_purchase_date)

### CLTV (Customer Lifetime Value)
- is a metric that represents the total financial value that customer brings to the company over the entire relationship with the bussiness.
- it takes into account not only the duration of customer's relationship with the company but also the monetary value of their specific transactions during that time.
- formula:
  CLTV = ((Average_Purchase_Value * Purchase_Frequency) / Churn Rate) * Profit Margin

  Where:
  
          Average_Purchase_Value = Total_Revenue / Total_Number_of_Orders
          
          Purchase_Frequency = Total_Number_of_Orders / Total_Number_of_Customers
          
          Churn_Rate = 1 / Average_Customer_Lifespan
          
          Profit_Margin = Net profit from a customer's purchase
  

In [1]:
# example of the calculation customer CLTV

import pandas as pd
data = {
    'customer_id': [1, 2, 3, 4, 5],
    'order_date': ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01'],
    'revenue': [100, 150, 200, 250, 300]
}

df = pd.DataFrame(data)
df['order_date'] = pd.to_datetime(df['order_date'])

# Calculate Time Frame
time_frame = (df['order_date'].max() - df['order_date'].min()).days / 30

# Calculate Revenue per Customer
revenue_per_customer = df.groupby('customer_id')['revenue'].sum()

# Calculate Average Purchase Frequency Rate
purchase_frequency = len(df) / len(df['customer_id'].unique())

# Calculate Average Purchase Value
average_purchase_value = df['revenue'].mean()

# Calculate Average Customer Lifespan
average_lifespan = time_frame / len(df['customer_id'].unique())

# Calculate CLTV
churn_rate = 1 / average_lifespan  # Assuming constant churn rate
profit_margin = 0.1  # Assuming 10% profit margin
cltv = ((average_purchase_value * purchase_frequency) / churn_rate) * profit_margin

print("Customer Lifetime Value (CLTV):", cltv)

Customer Lifetime Value (CLTV): 16.0


In [2]:
# example of the calculation customer Lifetime Span
data = {
    'customer_id': [1, 2, 3, 4, 5],
    'order_date': ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01'],
    'revenue': [100, 150, 200, 250, 300]
}

df = pd.DataFrame(data)
df['order_date'] = pd.to_datetime(df['order_date'])

# Calculate Customer Lifetime Span
customer_lifetime_span = df.groupby('customer_id')['order_date'].apply(lambda x: (x.max() - x.min()).days / 30)

print("Customer Lifetime Span:")
print(customer_lifetime_span)

Customer Lifetime Span:
customer_id
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: order_date, dtype: float64


In [3]:
import pandas as pd

df = pd.read_csv('../datasets/data_lifetime_CLTV.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df['Дата покупки'] = pd.to_datetime(df['Дата покупки'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Пользователь                    100 non-null    object        
 1   Дата покупки                    100 non-null    datetime64[ns]
 2   Сумма покупки                   100 non-null    int64         
 3   Категория продукта              100 non-null    object        
 4   Время с момента первой покупки  100 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.0+ KB


In [4]:
df

,Пользователь,Дата покупки,Сумма покупки,Категория продукта,Время с момента первой покупки
0,User1,2023-02-04,63,Home,25
1,User1,2023-01-18,149,Electronics,8
2,User1,2023-01-25,106,Home,15
3,User1,2023-01-30,193,Electronics,20
4,User1,2023-01-11,166,Clothing,1
...,...,...,...,...,...
95,User20,2023-01-15,48,Books,12
96,User20,2023-01-22,198,Home,19
97,User20,2023-01-25,44,Clothing,22
98,User20,2023-01-11,169,Home,8


In [27]:
# chat-gpt порекомендовал мне следующую формулу, которой я буду придерживаться
# cltv = average_purchase_value * purchase_frequency * average_customer_lifespan
# а по Lifetime я нашел следующую формулу: Customer_Lifetime = (last_purchase_date - first_purchase_date)

# Lifetime for each customer
# last_purchase = df.groupby('Пользователь')['Дата покупки'].max()
# first_purchase = df.groupby('Пользователь')['Дата покупки'].min()
# customer_lifetime_span = (last_purchase - first_purchase).dt.days

# But in this case we have a row (Время с момента первой покупки) that gives us the more accurate time user have been engaging the service
# So we can easily take a sum of all values related to each customer
# I will even provide a comparison of lifetime span calculated by date vs given values 
customer_lifetime_span = df.groupby(['Пользователь'])\
                           .agg({'Время с момента первой покупки': 'sum', 'Дата покупки': [('max-min', lambda x: x.max() - x.min())]})\
                           .reset_index()

# Average Customer Lifetime Span
avg_customer_lifetime_span = customer_lifetime_span['Время с момента первой покупки'].mean() # = 19.3

# Average Purchase Value
avg_purchase_value = df['Сумма покупки'].mean() # = 109.47

# Purchase frequency
purchase_freq = len(df) / len(df['Пользователь'].unique()) # = 5.0

cltv = avg_purchase_value * purchase_freq * avg_customer_lifetime_span

print(f"CLTV = {round(cltv['sum'], 2)} $\nMean customer lifetime span: {avg_customer_lifetime_span['sum']}")

CLTV = 39792.34 $
Mean customer lifetime span: 72.7


In [28]:
# Customer Lifetime Span
print(customer_lifetime_span)

   Пользователь Время с момента первой покупки Дата покупки
                                           sum      max-min
0         User1                             69      24 days
1        User10                             89      19 days
2        User11                            110      17 days
3        User12                             68      23 days
4        User13                            108      16 days
5        User14                             65      27 days
6        User15                             64      11 days
7        User16                             72      25 days
8        User17                             53      23 days
9        User18                             66      27 days
10       User19                             58      13 days
11        User2                             59      25 days
12       User20                             85      16 days
13        User3                             58      14 days
14        User4                         

In [19]:
df.groupby(['Пользователь']).agg({'Время с момента первой покупки': 'sum', 'Дата покупки': [('max-min', lambda x: x.max() - x.min())]}).reset_index()

,Пользователь,Время с момента первой покупки,Дата покупки
,,sum,max-min
0,User1,69,24 days
1,User10,89,19 days
2,User11,110,17 days
3,User12,68,23 days
4,User13,108,16 days
5,User14,65,27 days
6,User15,64,11 days
7,User16,72,25 days
8,User17,53,23 days


In [9]:
customer_lifetime_span

Пользователь
User1     24
User10    19
User11    17
User12    23
User13    16
User14    27
User15    11
User16    25
User17    23
User18    27
User19    13
User2     25
User20    16
User3     14
User4     15
User5     21
User6     18
User7     22
User8     13
User9     17
Name: Дата покупки, dtype: int64